In [1]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display
%matplotlib widget

In [47]:
class ShortDataDash:
    def __init__(self):
        self.df = pd.DataFrame()
        self.days_slider = widgets.IntSlider(
                                            value=5,
                                            min=5,
                                            max=252,
                                            step=1,
                                            description='Days Back',
                                            style={'description_width': 'initial'}
                                            )
        self.count_slider = widgets.IntSlider(
                                            value=10,
                                            min=1,
                                            max=25,
                                            step=1,
                                            description='Number to show.',
                                            style={'description_width': 'initial'}
                                            )
        self.output1 = widgets.Output()
        self.output2 = widgets.Output()
        self.load_button = widgets.Button(description="Load Data",
                                          layout=widgets.Layout(width='200px', height='40px'))
        self.load_button.on_click(self.load_button_click)
        self.show_button = widgets.Button(description="Change Number Shown",
                                          layout = self.load_button.layout)
        self.show_button.on_click(self.show_button_click)

        self.slider_box = widgets.HBox([self.days_slider, self.count_slider])
        self.button_box = widgets.VBox([self.load_button, self.show_button])

        self.stock_input = widgets.Text(
                                        value="GME",
                                        placeholder="GME",
                                        description='Ticker:',
                                    )
        self.ticker_button = widgets.Button(description="Plot Ticker")
        self.ticker_button.on_click(self.ticker_button_click)

    def show_button_click(self, b):
        self.output1.clear_output()
        with self.output1:
            self.update()

    def load_button_click(self,b):
        self.output1.clear_output()
        self.output2.clear_output()
        with self.output1:
            print(f"Data Loading for {self.days_slider.value} days")
            self.fetch_new_data()
            self.update()

    def ticker_button_click(self,b):
        self.output2.clear_output()
        with self.output2:
            self.ticker_plot()


    def fetch_new_data(self):
        self.df = pd.DataFrame()
        today = datetime.now().date()
        idx = 0
        len_df = 0
        while len_df < self.days_slider.value:
            date = today - timedelta(days = idx)
            r = requests.get(
                f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date.strftime('%Y%m%d')}.txt"
            )
            if r.status_code == 200:
                self.df = pd.concat([self.df,pd.read_csv(io.StringIO(r.text), sep = "|")], axis=0)
                len_df += 1
            idx +=1
        self.df = self.df[self.df.Date > 20100101]
        self.df.Date = self.df["Date"].apply(lambda x: datetime.strptime(str(x), "%Y%m%d"))


    def update(self):
        if not self.df.empty:
            temp = self.df.groupby("Symbol")[["ShortVolume","TotalVolume"]].agg("sum").sort_values(by="ShortVolume",ascending=False).head(self.count_slider.value)[::-1]
            self.fig,self.ax = plt.subplots(figsize = (6,6))
            self.ax.barh(temp.index, temp.TotalVolume, alpha = 0.4, label = "Total Volume")
            self.ax.barh(temp.index,temp.ShortVolume, label = "Short Volume")
            self.ax.set_title(f"Top {self.count_slider.value} Short Volume in Last {self.days_slider.value} Days")
            self.ax.legend()
            self.fig.tight_layout()
            plt.show()

    def ticker_plot(self):
        stock_data = self.df.copy().loc[self.df.Symbol == self.stock_input.value,["Date","ShortVolume","TotalVolume"]]
        self.fig2,self.ax2 = plt.subplots(figsize = (6,6))
        self.ax2.plot(stock_data.Date, stock_data.TotalVolume, alpha = 0.4, label = "Total Volume")
        self.ax2.plot(stock_data.Date,stock_data.ShortVolume, label = "Short Volume")
        self.ax2.set_title(f"Stock Volume and Short Volume for {self.stock_input.value.upper()}")
        self.ax2.legend()
        self.fig2.autofmt_xdate()
        self.fig2.tight_layout()
        plt.show()

    def build_app(self):
        title_html = """
<h2>Finra Short Data</h2>
<p>This widget downloads the consolidated NMS short data from FINRA and aggregates the data by summing over the entire time period.</p>
<p>Note that clicking the 'Load Data' button will reload all data.  This can get time consuming, so if you pick a few hundred days, expect a few minutes for loading time.</p>
"""
        middle_html = """
Here we allow the user to query for a single stock.  This will work with the loaded data.  Note that if you want to reload the data, this will once again take some time.
        """
        return [widgets.HTML(title_html,layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
                self.slider_box,
                self.button_box,
                self.output1,
                widgets.HTML(middle_html,layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
                self.stock_input,
                self.ticker_button,
                self.output2]

In [48]:
dash = ShortDataDash()
app = widgets.VBox(dash.build_app(), layout=widgets.Layout(max_width='1024px', margin='0 auto 0 auto'))

In [49]:
display(app)